## DB연결

In [1]:
from pymongo import MongoClient
# 네트워크 연결 규칙 : protocol://ip:port/path 
client = MongoClient('mongodb://192.168.0.50:27017/')

db = client["DB_SGMN"]
collection = db["COL_SCRAPPING_TOSS_COMMENT_HISTORY"]

In [2]:
getDatafromDB = collection.find()


data = list(getDatafromDB)


In [3]:
import pandas as pd

In [4]:
dfData = pd.DataFrame(data)

In [ ]:
dfData.info()

In [ ]:
dfDataEdited = dfData.drop(labels='_id',axis=1)
dfDataEdited

In [7]:
dataToDicOnlyCommentAndDate = dfDataEdited[dfDataEdited['SYMBOL'] == 'TSLA'][['COMMENT', 'DATE', 'SYMBOL']].to_dict(orient="list")


In [ ]:
commentDatasList = dfDataEdited.groupby('DATE')['COMMENT'].apply(list).to_dict()
commentDatasList
type(commentDatasList)

In [ ]:
dateList = dataToDicOnlyCommentAndDate['DATE']
commentList = dataToDicOnlyCommentAndDate['COMMENT']
# dateList
type(dateList)

In [ ]:
commentList
len(commentList)

## TF-IDF 사용 문장 유사도 작성

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer # Word Embedding
# tfidfVectorizer = TfidfVectorizer()
tfidfVectorizer = TfidfVectorizer()
tfidfVectorizer.fit(commentList) # 학습용
result_vectors = tfidfVectorizer.transform(commentList) # 서비스용


In [ ]:
result_vectors.toarray().shape

In [ ]:
result_vectors.toarray().shape, tfidfVectorizer.get_feature_names_out().shape

In [14]:
vectors_100 = result_vectors[:100]

In [ ]:
## 문장간 유사도 측정

from sklearn.metrics.pairwise import cosine_similarity
df_similarity = cosine_similarity(vectors_100, vectors_100)
df_similarity

In [ ]:
import pandas as pd

pd.DataFrame(data=df_similarity

, index=commentList[:100]
, columns=commentList[:100]
)

In [17]:
import pickle

# 데이터 저장하기
data_to_save = {
    'similarity_matrix': df_similarity,
    'comments': commentList[:100],  # 또는 [commentList[i] for i in random_indices]
    'vectors': vectors_100
}

# 저장
with open('similarity_data.pkl', 'wb') as f:
    pickle.dump(data_to_save, f)

# 나중에 불러올 때
with open('similarity_data.pkl', 'rb') as f:
    loaded_data = pickle.load(f)

# 불러온 데이터 사용
similarity_matrix = loaded_data['similarity_matrix']
comments = loaded_data['comments']
vectors = loaded_data['vectors']

# DataFrame 재생성
df_similarity_matrix = pd.DataFrame(
    data=similarity_matrix,
    index=comments,
    columns=comments
)
